# Detection of Master Regulators on a dummy example

This vignette displays some examples of what can be achieved using **NORDic** in order to detect master regulators.

## Installation

In [1]:
import NORDic
! pip freeze | grep "NORDic"

-e git+https://github.com/clreda/nordic.git@859bd0341858399c88ca776d508024c9ed4c66c5#egg=NORDic


## Detection of master regulators

**NORDic PMR** exploits a Boolean network in order to find master regulators. Its approach is based on influence maximization. For more information about the method, please have a look at the following [reference](https://clreda.github.io/assets/reda2022prioritization_accepted.pdf):


> Réda, C., & Delahaye-Duriez, A. (2022). Prioritization of Candidate Genes Through Boolean Networks. In *International Conference on Computational Methods in Systems Biology* (pp. 89-121). Springer, Cham.


The input network does not need to be an output of **NORDic NI**, and the corresponding .BNET file can be independenty generated. We illustrate this by feeding to **NORDic PMR** a dummy example.

In [2]:
from subprocess import call as sbcall
save_folder="MODELS/dummy/"
sbcall("mkdir -p "+save_folder, shell=True)

0

### A. Creating the dummy network example

The dummy example has 11 genes named X0, X1, ..., X10 (with one isolated gene X10), and 20 edges. We expect X0 and X1 to be  master regulators, and X10 to have no influence on the network (because it is isolated). Let us test this using **NORDic PMR**.

In [3]:
import pandas as pd
import numpy as np

## List of input nodes for each edge
source = list(map(lambda x: "X%d" % x,[0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,3,4,5]))
## List of target nodes for each edge
target = list(map(lambda x : "X%d" % x,[2,3,4,5,6,7,8,9,2,3,4,5,6,7,8,9,6,7,8,9]))

## List of all genes
genes = list(sorted(list(set(source+target))))+["X10"]

## Create the network (adjacency matrix)
influences = np.zeros((len(genes), len(genes)))
for s, t in zip(source, target):
    influences[genes.index(s)][genes.index(t)] = 1
influences = pd.DataFrame(influences, index=genes, columns=genes).astype(int)
influences

X0  X1  X2  X3  X4  X5  X6  X7  X8  X9  X10
X0    0   0   1   1   1   1   1   1   1   1    0
X1    0   0   1   1   1   1   1   1   1   1    0
X2    0   0   0   0   0   0   1   0   0   0    0
X3    0   0   0   0   0   0   0   1   0   0    0
X4    0   0   0   0   0   0   0   0   1   0    0
X5    0   0   0   0   0   0   0   0   0   1    0
X6    0   0   0   0   0   0   0   0   0   0    0
X7    0   0   0   0   0   0   0   0   0   0    0
X8    0   0   0   0   0   0   0   0   0   0    0
X9    0   0   0   0   0   0   0   0   0   0    0
X10   0   0   0   0   0   0   0   0   0   0    0

Let us plot the network using **NORDic** functions, and save the corresponding image in *save_folder*/example.png. Parameter *optional* traces edges with dashed lines when set to True, resp. with solid lines when set to False.

In [4]:
from NORDic.UTILS.utils_plot import influences2graph

influences2graph(influences, save_folder+"example", optional=False, 
                 compile2png=True, engine="sfdp")
from IPython.display import Image
Image(filename=save_folder+'example.png') 

Now that we have built the topology of the network, its dynamics remain to be defined. In that case, we assume that all activators (i.e., regulators connected with an activatory edge) should be active (i.e., an "active" expression level) for the regulated gene to be active.

In [5]:
## Gene regulatory functions (for each node in the network)
grfs = {}
for si, s in enumerate(source):
    t = target[si]
    grf = grfs.get(t, [])
    grfs.update(dict([[t, list(set(grf+[s]))]]))

Now, let us write down the corresponding .BNET file *save_folder*/example.bnet

In [6]:
with open(save_folder+"example.bnet", "w") as f:
    network = []
    for g in genes:
        if (g in grfs):
            network += [g+", "+"&".join(grfs[g])]
        else:
            network += [g+", 1"]
    f.write("\n".join(network))
    
## Print the resulting network
with open(save_folder+"example.bnet", "r") as f:
    network = f.read()
print("\n".join([", ".join(s.split(", ")) for s in network.split("\n")]))

X0, 1
X1, 1
X2, X0&X1
X3, X0&X1
X4, X0&X1
X5, X0&X1
X6, X0&X1&X2
X7, X0&X1&X3
X8, X4&X0&X1
X9, X5&X0&X1
X10, 1


### B. Creating random "patient" samples

In **NORDic PMR**, the ability of a given gene to disrupt the expression of the whole network is tested in a specific disease-related transcriptional context, i.e., patient transcriptional profiles ("states"). Since here the nodes in the network are not related to actual genes, we generate 100 binary states at random. Those states should be a dataframe, with samples/patients in columns, genes in rows.

In [7]:
state_len = 200
states = pd.DataFrame(
  [np.random.choice([0,1], p=[0.5,0.5], size=len(genes)).tolist() for _ in range(state_len)]
  , columns=genes, index=range(state_len)).T
states

0    1    2    3    4    5    6    7    8    9    ...  190  191  192  \
X0     1    1    0    1    0    0    0    0    1    1  ...    1    1    0   
X1     0    0    1    0    1    0    0    0    0    1  ...    0    0    0   
X2     0    0    1    0    1    1    1    1    1    0  ...    0    1    1   
X3     0    0    1    1    1    1    0    0    0    1  ...    0    1    0   
X4     1    1    1    0    1    1    0    0    0    0  ...    1    1    1   
X5     1    1    0    1    0    0    0    0    1    1  ...    1    1    1   
X6     1    1    1    0    1    1    0    0    1    1  ...    0    0    0   
X7     1    1    0    1    1    1    0    1    0    0  ...    1    1    0   
X8     1    1    0    1    1    0    1    0    1    1  ...    0    1    1   
X9     0    0    1    1    1    1    0    1    0    0  ...    1    0    0   
X10    0    1    1    0    0    0    1    1    1    1  ...    1    1    1   

     193  194  195  196  197  198  199  
X0     0    0    1    0    0    1    1  
X1     1    0    1    1    1    0    0  
X2     0    1    1    1    0    0    0  
X3     0    0    1    0    1    0    1  
X4     0    0    0    0    0    1    0  
X5     0    0    0    1    0    1    1  
X6     0    0    1    0    1    0    1  
X7     0    1    0    1    1    0    0  
X8     0    0    1    0    0    1    1  
X9     0    1    0    1    1    0    1  
X10    0    0    0    0    1    0    1  

[11 rows x 200 columns]

### C. Selection of parameters

Again, we need to set the random seed for increased reproducibility, and set the maximum number of threads to use:

In [8]:
seed_number=0
from multiprocessing import cpu_count
njobs=min(5,max(1,cpu_count()-2)) #minimum between all available -2 and 5

In order to prove our conjecture (that X0 and X1 are the most regulatory in this network), we set k to 2, that is, we return all the sets of 2 nodes in the network that *roughly* (see paper) maximize the perturbation of the whole network.

In [9]:
k=2

The perturbation of the network should be computed for each gene in the network (list *gene_inputs*), on every "regulatable" gene (that is, a gene which has at least one regulator which is not itself) (list *genes_outputs*).

In [10]:
## Compute the list of "regulatable" genes from the network
with open(save_folder+"example.bnet", "r") as f:
    network = f.read()
gene_outputs = [ x.split(", ")[0] for x in network.split("\n")[:-1] 
    if (x.split(", ")[1] not in [x.split(", ")[0], "0", "1"])]

IM_params = {
    "seed": seed_number, "gene_inputs": genes, # genes to be perturbed
    "gene_outputs": gene_outputs # genes to be observed
}

Let us now set parameters relative to the simulations of trajectories under perturbation of the network (see [mpbn-sim](https://github.com/bnediction/mpbn-sim)):

In [11]:
SIMU_params = {
    'nb_sims': 1000, 'rates': "fully_asynchronous", 'thread_count': njobs,
    'depth': "constant_unitary",
}

All in all

In [12]:
seed_number=12345
from multiprocessing import cpu_count
njobs=min(5,max(1,cpu_count()-2))

k=2

with open(save_folder+"example.bnet", "r") as f:
    network = f.read()
gene_outputs = [ x.split(", ")[0] for x in network.split("\n")[:-1] 
    if (x.split(", ")[1] not in [x.split(", ")[0], "0", "1"])]

IM_params = {
    "seed": seed_number, "gene_inputs": genes,
    "gene_outputs": gene_outputs
}

SIMU_params = {
    'nb_sims': 1000, 'rates': "fully_asynchronous", 'thread_count': njobs,
    'depth': "constant_unitary",
}

### D. Detection of master regulators

Function *greedy* implements the approximately optimal greedy algorithm from the following paper

>Kempe, David, Jon Kleinberg, and Éva Tardos. "Maximizing the spread of influence through a social network." Proceedings of the ninth ACM SIGKDD international conference on Knowledge discovery and data mining. 2003.

It returns the predicted set of master regulators, along with the *spread values* computed at each step of the algorithm for each node. Spread values are correlated with the ability to disrupt the expression of the network (see paper).

In [13]:
from NORDic.NORDic_PMR.functions import greedy

S, spreads = greedy(save_folder+"example.bnet", k, states, IM_params, SIMU_params, 
                    save_folder=save_folder)

<NORD_PMR> Iteration k=1
	<NORD_PMR> Initial state 4 (gene(s):X0)
	<NORD_PMR> Initial state 1 (gene(s):X0)
	<NORD_PMR> Initial state 0 (gene(s):X0)
	<NORD_PMR> Initial state 3 (gene(s):X0)
	<NORD_PMR> Initial state 2 (gene(s):X0)
1 wild type attractors with proba > 0 (initial state 4)
Gene X0 (1/11) in state 4 - constant_unitary_depth{}	fully_asynchronous_rates{}
1 wild type attractors with proba > 0 (initial state 0)
1 wild type attractors with proba > 0 (initial state 1)
Gene X0 (1/11) in state 0 - constant_unitary_depth{}	fully_asynchronous_rates{}
1 wild type attractors with proba > 0 (initial state 2)
1 wild type attractors with proba > 0 (initial state 3)
Gene X0 (1/11) in state 1 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X0 (1/11) in state 2 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X0 (1/11) in state 3 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X1 (2/11) in state 4 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X1 (2

100%|██████████| 1/1 [00:00<00:00, 50533.78it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 61680.94it/s]

100%|██████████| 1/1 [00:00<00:00, 55188.21it/s]

Gene X9 (10/11) in state 0 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X10 (11/11) in state 4 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X9 (10/11) in state 2 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X9 (10/11) in state 3 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X9 (10/11) in state 1 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X10 (11/11) in state 0 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X10 (11/11) in state 2 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X10 (11/11) in state 3 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X10 (11/11) in state 1 - constant_unitary_depth{}	fully_asynchronous_rates{}
	<NORD_PMR> Initial state 5 (gene(s):X0)
	<NORD_PMR> Initial state 6 (gene(s):X0)
1 wild type attractors with proba > 0 (initial state 5)
Gene X0 (1/11) in state 5 - constant_unitary_depth{}	fully_asynchronous_rates{}
	<NORD_PMR> Initial state 7 (gene(s):X0)
	<NORD_


100%|██████████| 1/1 [00:00<00:00, 64527.75it/s]

100%|██████████| 1/1 [00:00<00:00, 61680.94it/s]

100%|██████████| 1/1 [00:00<00:00, 59074.70it/s]

Gene X4 (5/11) in state 6 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X7 (8/11) in state 5 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X4 (5/11) in state 7 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X4 (5/11) in state 8 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X5 (6/11) in state 9 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X8 (9/11) in state 5 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X5 (6/11) in state 6 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X5 (6/11) in state 7 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X6 (7/11) in state 9 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X5 (6/11) in state 8 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X9 (10/11) in state 5 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X6 (7/11) in state 6 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X6 (7/11) in state 7 - constant_un

...


100%|██████████| 1/1 [00:00<00:00, 51781.53it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s] 74626.43it/s]

100%|██████████| 1000/1000 [00:00<00:00, 58975.86it/s]

	<NORD_PMR> Initial state 194 (gene(s):X0)
Gene X7 (7/10) in state 190 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X3 (3/10) in state 192 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X5 (5/10) in state 191 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X3 (3/10) in state 193 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X4 (4/10) in state 192 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X8 (8/10) in state 190 - constant_unitary_depth{}	fully_asynchronous_rates{}
1 wild type attractors with proba > 0 (initial state 194)
Gene X0 (1/10) in state 194 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X6 (6/10) in state 191 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X5 (5/10) in state 192 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X4 (4/10) in state 193 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X9 (9/10) in state 190 - constant_unitary_depth{}	fully_asynchronous_rates

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 59918.63it/s]7it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

1 wild type attractors with proba > 0 (initial state 197)
Gene X0 (1/10) in state 197 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X5 (5/10) in state 195 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X3 (3/10) in state 196 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X10 (10/10) in state 194 - constant_unitary_depth{}	fully_asynchronous_rates{}
	<NORD_PMR> Initial state 198 (gene(s):X0)
Gene X6 (6/10) in state 195 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X2 (2/10) in state 197 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X4 (4/10) in state 196 - constant_unitary_depth{}	fully_asynchronous_rates{}
	<NORD_PMR> Initial state 199 (gene(s):X0)
Gene X7 (7/10) in state 195 - constant_unitary_depth{}	fully_asynchronous_rates{}
1 wild type attractors with proba > 0 (initial state 198)
Gene X5 (5/10) in state 196 - constant_unitary_depth{}	fully_asynchronous_rates{}
Gene X3 (3/10) in state 197 - constant_unitary_depth{}	ful

100%|██████████| 1000/1000 [00:00<00:00, 59717.30it/s]

### E. Checking out the results

Let us find out if X0 and X1 are indeed the most regulatory in the network. In resulting set S, nodes are contained in the same bin if they yield a similar spread value. Then, we need to unfold the nested list S:

In [14]:
unfoldS = [x for s in S for x in s]
assert all([s in ["X0","X1"] for s in unfoldS])

So, indeed, the conjecture checks out. Now, let us have a look at the spread values:

In [15]:
spreads

[['X1']]  [['X1'], ['X0']]
X0   0.394744          0.652901
X1   0.409321               NaN
X2   0.068331          0.459109
X3   0.066906          0.457162
X4   0.066906          0.455218
X5   0.070473          0.464966
X6   0.000000          0.409321
X7   0.000000          0.409321
X8   0.000000          0.409321
X9   0.000000          0.409321
X10  0.000000          0.409321

Genes are in rows, the current set of master regulators (for i=1, 2, ..., k) is shown in columns. 

- The first column corresponds to spread values computed when perturbing the set {g} of genes, where g is the gene shown in rows. X0 is the only maximizer of the spread value for i=1, so the current set of master regulators at i=1 is [X0].

- The second column corresponds to spread values computed when perturbing the set {g, X0} of genes, where g is the gene shown in rows. Note that the presence of the NaN is due to the fact that we have already tested X0 alone in the previous step. X1 is the only maximizer of the spread value for i=2, so the current set of master regulators at i=2 is [X0, X1].

- Since we have reached i=2=k, the algorithm stops, and returns [X0, X1].